In [1]:
%%capture

# get_corpus_path
# get_txt_orig_path

%run ../path_manager.ipynb


# detect_acronyms
# detect_acronyms_from_file
# merge_corpora_acronyms_map
# get_corpus_top_acronym_prototypes
%run ./AcronymModule.ipynb

In [2]:
import pandas as pd
import re
from collections import Counter
import glob
import os
from joblib import Parallel, delayed

Please replace the `sample_file` parameter to adapt to your file path.

This method is applied to the original files prior to cleaning.

In [3]:
# sample_file = os.path.join(get_txt_orig_path('WB'), 'wb_30385515.txt')

# with open(sample_file, 'rb') as fl:
#     txt = fl.read()
#     txt = re.sub(b'[\r\n]+', b' ', txt)
#     txt = txt.decode('utf-8', errors='ignore')

# acronyms_map = detect_acronyms(txt)

# for a, p in acronyms_map.items():
#     print(a)
#     for i in p:
#         print(f'\t{i}')
#     print()

# Detect acronyms for corpus ids

In [4]:
N_JOBS = 48
corpus_ids = ['IMF']

In [5]:
# %%time
# corpora_acronyms = {}

# for corpus_id in corpus_ids:
#     print(f'Detecting acronyms for corpus {corpus_id}...')
    
#     corpus_full_acronyms_map = {}
#     fpath_iterator = glob.iglob(os.path.join(get_txt_orig_path(corpus_id), f'{corpus_id.lower()}_*.txt'))
    
#     acronyms_maps = Parallel(n_jobs=N_JOBS)(delayed(detect_acronyms_from_file)(fpath) for fpath in fpath_iterator)
    
#     for acronyms_map in acronyms_maps:
#         if acronyms_map:
#             for a, p in acronyms_map.items():
#                 if a in corpus_full_acronyms_map:
#                     for i in p:
#                         if i in corpus_full_acronyms_map[a]['prototypes']:
#                             corpus_full_acronyms_map[a]['prototypes'][i] += 1
#                         else:
#                             corpus_full_acronyms_map[a]['prototypes'][i] = 1
#                     corpus_full_acronyms_map[a]['doc_freq'] += 1
#                 else:
#                     corpus_full_acronyms_map[a] = {'prototypes': {i: 1 for i in p}, 'doc_freq': 1}      
                    
#     corpora_acronyms[corpus_id] = corpus_full_acronyms_map

In [6]:
%%time
corpora_acronyms = {}

for corpus_id in corpus_ids:
    print(f'Detecting acronyms for corpus {corpus_id}...')
    
    corpus_full_acronyms_map = {}
    fpath_iterator = glob.iglob(os.path.join(get_txt_orig_path(corpus_id), f'{corpus_id.lower()}_*.txt'))
    
    acronyms_maps = Parallel(n_jobs=N_JOBS)(delayed(detect_acronyms_from_file)(fpath) for fpath in fpath_iterator)
    
    for acronyms_map in acronyms_maps:
        if acronyms_map:
            for a, p in acronyms_map.items():
                if a in corpus_full_acronyms_map:
                    for i in p:
                        if i in corpus_full_acronyms_map[a]['prototypes']:
                            corpus_full_acronyms_map[a]['prototypes'][i] += 1
                        else:
                            corpus_full_acronyms_map[a]['prototypes'][i] = 1
                    corpus_full_acronyms_map[a]['doc_freq'] += 1
                else:
                    corpus_full_acronyms_map[a] = {'prototypes': {i: 1 for i in p}, 'doc_freq': 1}      
                    
    corpora_acronyms[corpus_id] = corpus_full_acronyms_map

Detecting acronyms for corpus IMF...
CPU times: user 8.34 s, sys: 1.58 s, total: 9.92 s
Wall time: 6min 58s


In [7]:
sorted(corpora_acronyms[corpus_id].items(), key=lambda x: -x[1]['doc_freq'])[:1]

[('IMF',
  {'doc_freq': 3795,
   'prototypes': {'F Immcial Manual': 1,
    'Financiers Institutions Microfinance': 1,
    'Four Including Multimod': 1,
    'Institutions Micro Finance': 4,
    'Institutions Micro- Finance': 1,
    'Intemationai Monetary Fund': 1,
    'Intemational Monetaly Fund': 1,
    'Intemational Monetary F': 1,
    'Intemational Monetary Fund': 5,
    'Intemtional Monetary Fund': 1,
    'Inter- Monetary Fund': 2,
    'Interna- Monetary Fund': 2,
    'Internationai Monetary Fund': 2,
    'Internationai Monetaw Fund': 1,
    'International Mon y Fund': 1,
    'International Mone- Fund': 1,
    'International Monetaiy Fund': 1,
    'International Monetaly Fund': 2,
    'International Monetary F': 7,
    'International Monetary Fund': 3748,
    'International Monetary Funds': 1,
    'International Monetaty Fund': 5,
    'International Monetazy Fund': 1,
    'International Monetmy Fund': 1,
    'Internationat Monetary Fund': 3}})]

# Extract the top prototypes for each acronym

In [8]:
acronyms_popular_prototype = get_corpus_top_acronym_prototypes(corpora_acronyms[corpus_id])

In [9]:
acronyms_popular_prototype.head(5)

,acronym,doc_freq,full_name,percentage
0,IMF,3795,International Monetary Fund,0.987615
1,IMF,3795,Institutions Micro Finance,0.001054
2,IMF,3795,International Monetaty Fund,0.001318
3,IMF,3795,International Monetary F,0.001845
4,IMF,3795,Intemational Monetary Fund,0.001318


UNO, FAO, IMF, etc (or to convert into tokens/ngrams). Have you ever found a tool that would help detect them (if we can detect them, we can decide what to do with them)? This may be a bit tricky in some cases (e.g., many of our documents probably mention the World Health Organization as WHO)

In [10]:
sample_set = ['UNO', 'FAO', 'IMF', 'WHO', 'PPP']

In [11]:
acronyms_popular_prototype[acronyms_popular_prototype.acronym.isin(sample_set)].sort_values(['doc_freq', 'percentage'], ascending=False)

,acronym,doc_freq,full_name,percentage
0,IMF,3795,International Monetary Fund,0.987615
3,IMF,3795,International Monetary F,0.001845
2,IMF,3795,International Monetaty Fund,0.001318
4,IMF,3795,Intemational Monetary Fund,0.001318
1,IMF,3795,Institutions Micro Finance,0.001054
164,PPP,569,Purchasing Power Parity,0.632689
160,PPP,569,Public Private Partnership,0.158172
161,PPP,569,Public Private Partnerships,0.054482
162,PPP,569,Purchasing Power Parities,0.022847
163,PPP,569,Public- Private Partnership,0.019332


In [12]:
acronyms_popular_prototype[acronyms_popular_prototype.acronym == 'CRRA']

,acronym,doc_freq,full_name,percentage
3238,CRRA,36,Constant Relative Risk Aversion,0.972222
3239,CRRA,36,Cient of Relative Risk Aversion,0.027778


# Store detected prototypes per corpus

In [14]:
MIN_DOC_FREQ = 10

for corpus_id in corpus_ids:
    acronyms_popular_prototype = get_corpus_top_acronym_prototypes(corpora_acronyms[corpus_id])
    
    acronyms_popular_prototype[acronyms_popular_prototype.doc_freq > MIN_DOC_FREQ].sort_values(['doc_freq', 'percentage'], ascending=False).to_csv(
        os.path.join(get_corpus_path(corpus_id), f'{corpus_id.lower()}_detected_acronyms_prototypes.csv'),
        index=None
    )

In [13]:
acronyms_popular_prototype[acronyms_popular_prototype.doc_freq > 50].doc_proportion_1.describe()

count    828.000000
mean       0.355177
std        0.274066
min        0.004622
25%        0.117647
50%        0.267342
75%        0.573590
max        0.971014
Name: doc_proportion_1, dtype: float64

In [14]:
acronyms_popular_prototype[
    (acronyms_popular_prototype.doc_freq > 50) &
    (acronyms_popular_prototype.doc_proportion_1 < 0.15)
]

,acronym,doc_freq,popular_prototype_1,doc_proportion_1,popular_prototype_2,doc_proportion_2,popular_prototype_3,doc_proportion_3,popular_prototype_4,doc_proportion_4,popular_prototype_5,doc_proportion_5
1,FAD,2056,Fiscal Affairs Department,0.139591,Saeed (MCM) Cevik,0.003405,Public Ms Budina,0.002918,Santiago Acosta Ormaechea,0.002918,Salvatore Dell Erba,0.002918
2,RHS,1947,In (LHS) In,0.004622,Exports Imports Current,0.004622,Effective Exchange Rate,0.003595,Baseline Real Gross,0.003082,Rate Daily Intervention,0.002568
5,SPR,1487,(AFR) Peter Allum,0.016812,Policy Review Department,0.015467,(WHD) Peter Allum,0.010760,(WHD) Bob Traa,0.010760,(AFR) Dhaneshwar Ghura,0.009415
8,FDI,1383,Foreign Direct Investment,0.088937,Privatization Private Foreign,0.009400,Foreign Direct Investments,0.008677,Abroad In Indonesia,0.004338,The The The,0.004338
10,REER,1255,Real Effective Exchange Rate,0.058964,Real Effective Exchange Rates,0.006375,The Real Background The,0.005578,Exchange Rate Assessment The,0.003984,(RCI) Exports Exports Imports,0.003984
11,MCM,1252,Capital Markets Department,0.103035,Monetary Capital Markets,0.023163,Capital Market Department,0.004792,(MCD) Miguel Savastano,0.004792,Gottschalk (FAD) Song,0.003994
12,OED,1248,Offices Executive Directors,0.006410,Jouahri Mr Da,0.005609,Mr Abradu Otoo,0.004808,Reps Mr Cardoso,0.004808,Operations Evaluation Department,0.004808
16,PPP,1174,Income Malnutrition Poverty,0.076661,Public Private Partnership,0.071550,Purchasing Power Parity,0.046848,Public Private Partnerships,0.030664,Due Deposits Preprovision,0.011073
17,VAT,1111,Value Added Tax,0.134113,Expert Late Tax,0.009001,Enterprise Social Value,0.007201,The The The,0.005401,Noncustoms Income Sales,0.005401
20,PDR,1000,Development Review Department,0.006000,Mr Mc Gettigan,0.004000,Mr Op Beke,0.004000,(FAD) Mr Hofman,0.004000,(FAD) Mr Kelmanson,0.004000


In [15]:
corpora_acronyms[corpus_id]['PPP']

{'doc_freq': 1174,
 'prototypes': {'(ANNP) At This': 1,
  '(BUILD) Public Staffs': 1,
  '(CAF) (FAB) (MMA)': 1,
  '(CDMT) Par Gouvernement': 1,
  '(CSO) Es Staff': 1,
  '(DSNCRP) This Moreover': 1,
  '(ECM) With According': 1,
  '(EREER) These (UIP)': 1,
  '(GDP) Capital Relative': 2,
  '(GFCF) The Es': 1,
  '(GRS) The Parliament': 1,
  '(HII) Expropriation Fund': 2,
  '(MB) (ES) (ERER)': 1,
  '(MB) The The': 1,
  '(NATREX) Stein Allen': 1,
  '(OHADA) Tribunal Com': 1,
  '(OMD) Cameroun Dans': 1,
  '(RER) (REER) The': 1,
  '(RER) Indeed Hrv': 1,
  '(ROW) We Ps': 1,
  '(TSA) June However': 1,
  'Abstract Long The': 1,
  'Abstract This Africa': 1,
  'According (LOOP) At': 2,
  'According Slovakia The': 1,
  'Accordingly In Such': 1,
  'Accordingly Staff In': 1,
  'Ackland Dowrick Freyens': 1,
  'Act This For': 1,
  'Africa Furthermore Following': 1,
  'Africa In Europe': 1,
  'Africa Saharan African': 1,
  'After Es Private': 1,
  'After Next In': 1,
  'All Table The': 1,
  'Also In Chap

# Unify corpus ids acronyms maps

In [15]:
merged_acronyms_map = merge_corpora_acronyms_map(list(corpora_acronyms.values()))

In [16]:
merged_acronyms_map['PPP']

{'doc_freq': 787,
 'prototypes': {'(ANNP) At This': 1,
  '(BUILD) Public Staffs': 1,
  '(CAF) (FAB) (MMA)': 1,
  '(CDMT) Par Gouvernement': 1,
  '(CSO) Es Staff': 1,
  '(ECM) With According': 1,
  '(EREER) These (UIP)': 1,
  '(GDP) Capital Relative': 1,
  '(GFCF) The Es': 1,
  '(HII) Expropriation Fund': 2,
  '(MB) (ES) (ERER)': 1,
  '(MB) The The': 1,
  '(NATREX) Stein Allen': 1,
  '(OHADA) Tribunal Com': 1,
  '(RER) (REER) The': 1,
  '(ROW) We Ps': 1,
  'Access Gini Poverty': 1,
  'According (LOOP) At': 2,
  'According Slovakia The': 1,
  'Accordingly In Such': 1,
  'Accordingly Staff In': 1,
  'Ackland Dowrick Freyens': 1,
  'Act This For': 1,
  'Africa Furthermore Following': 1,
  'Africa In Europe': 1,
  'After Es Private': 1,
  'After Next In': 1,
  'All Table The': 1,
  'Alternatives Country (REER)': 1,
  'Although It First': 1,
  'American Chinese American': 1,
  'Among Cottani They': 1,
  'Among Weaknesses These': 1,
  'Analizat Shumica Kosov': 1,
  'Analytical Assuming The': 